In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
       
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_columns', 60)
pd.set_option('display.max_rows', 500)
import warnings
warnings.filterwarnings('ignore')

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

#### A introduction to blending in models. 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier
import xgboost as xgb
from catboost import CatBoostClassifier

from sklearn.model_selection import GridSearchCV , RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss

In [ ]:
train_df = pd.read_csv(os.path.join(dirname,'train.csv'))
test_df = pd.read_csv(os.path.join(dirname,'test.csv'))

In [ ]:
features = train_df.columns[1:-1]
known_x = train_df[features]
known_y = train_df[train_df.columns[-1]]
test_x = test_df[features]

In [ ]:
train_x,val_x, train_y, val_y = train_test_split(known_x,known_y,train_size=0.8,random_state = 0)

### Average Blending

#### CatBoost

In [ ]:
cat1 =  CatBoostClassifier(iterations=1500,learning_rate = 0.1, max_depth = 4,l2_leaf_reg = 7,
                           loss_function = 'MultiClass' ,eval_metric = 'MultiClass',random_seed = 0,verbose = 0)

In [ ]:
cat1.fit(train_x, train_y, eval_set = (val_x,val_y))

In [ ]:
stage1_pred_cat = cat1.predict_proba(test_x)

#### XGB

In [ ]:
xgb1 = XGBClassifier(subsample= 0.6,reg_alpha= 0.1,min_child_weight= 3,max_depth= 6,
            learning_rate= 0.03,gamma= 0.0,colsample_bytree= 0.6, seed=27)

In [ ]:
xgb1.fit(known_x, known_y,eval_metric='logloss')
stage1_pred_xgb = xgb1.predict_proba(test_x)

#### Blending

In [ ]:
stage1_cat_df = pd.DataFrame(stage1_pred_cat, columns = cat1.classes_)
stage1_xgb_df = pd.DataFrame(stage1_pred_xgb, columns = xgb1.classes_)

In [ ]:
blend_df = (stage1_cat_df*0.8) + (stage1_xgb_df*0.2) 

In [ ]:
test_df[blend_df.columns] = blend_df

In [ ]:
submit_df = test_df[['id','Class_1','Class_2','Class_3','Class_4']]
submit_df.to_csv('avg_blend_CatXgb_submission.csv', index = False)